# SQL and Databases

What is a database? 
- A database is a collection of information that is organized so that it can be easily accessed, managed, and updated.
- In a relational database, digital information about a specific customer is organized into rows, columns, and tables which are indexed to make it easier to find relevant information through SQL queries.
- Source: https://searchsqlserver.techtarget.com/definition/database


Why create a relational database?
- Flexible, can add new data and new relationships easily
- Eliminates redundancies (e.g., a csv with several rows of repeated data)
- Source: https://www.ibm.com/cloud/learn/relational-databases
- Example: https://www.pragimtech.com/blog/contribute/article_images/2220211210231003/what-is-a-relational-database.jpg

What is ORM? 
- ORM stands for Object Relational Mapping. 
- ORM is a technique used in creating a "bridge" between OOP and relational databases. 
- An ORM tool is a software designed to help OOP (like Python) to interact with relational databases. 
<br>
<br>

#### Python offers several ORM tools that help us to do that! 

* Here is an example of SQL code that retrieves information about a particular user from a database: 

`SELECT id, name, email, country, phone_number`

`FROM users`

`WHERE id = 20`

* This code returns specified information about this user from a table called `users`, using the `WHERE` clause where we specified that `id = 20` for this user.
* On the other hand, an ORM tool can do the exact same query with much shorter code! 
`users.GetById(20)`

##### Let's install `sqlite` first! 

- On **Mac**: open terminal, then run `brew install sqlite3`
- On **Windows**: follow https://www.guru99.com/download-install-sqlite.html

##### Now we can install this library `sqlalchemy` for python. 

In [1]:
# !pip3 install sqlalchemy

- Check tutorial: http://pythoncentral.io/introductory-tutorial-python-sqlalchemy/
- Find documentation: https://www.kite.com/python/docs/

In [2]:
import sqlalchemy

# fun trick!
import warnings
warnings.filterwarnings('ignore')

#### 1. Establish connectivity using the Engine 

* The engine is typically a global object created using `create_engine()` just once for a particular database.

In [3]:
# Make sure to change to your directory
engine = sqlalchemy.create_engine('sqlite:///players.db')

* The parameter `echo = True` makes SQLAlchemy log all SQL commands it is doing while you apply commands.
* For now, I'm just turning it off to make the cell output cleaner. 

The database will be intact, until you execute the commands. 

#### 2. Instantiate a Base

* With the help of ORM tools, we don't need to write SQL queries anymore.
* Everything is done using a declarative system.
* The `Base` class maintains a catalog of all the tables and classes (i.e. structures of the database). 

More information: https://stackoverflow.com/questions/1279613/what-is-an-orm-how-does-it-work-and-how-should-i-use-one

In [4]:
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base() 

* In our database, each class is a table. Each attribute is a column in the table. 

##### Table relations in SQL

1. One-to-one: a record in one table is linked to max one record in another table. 
    - A student can have only one student ID and only one grade for a particular course. 
2. One-to-many: a single record in one table can be mapped to multiple rows of another table. 
    - A student can have multiple email addresses of different types, e.g. school email, personal email, etc. 
3. Many-to-many: each row in a table can map to multiple rows in another table and vice versa. 
    - A student can know how to use several programming languages like R, Python, Julia. 
    - And one programming language (e.g. Python) can be mapped to multiple students who have the same skills. 

A **primary key** uniquely identifies a record (row/orbservation) in a table. A **foreign key** links the record in a table to one in another table.

#### 3. Create our tables

##### Let's create a one-to-many example

* Basketball teams can have many players in each, but one player can only play for one basketball team. 

We need: 
1. Two tables: one for **players**, and one for **teams**
2. Foreign key on **players** to reference unique team IDs (primary key on **teams**)
3. Use `relationship()` method to specify 

In [5]:
from sqlalchemy import Column, Integer, String, ForeignKey

##### Defining the `Player` table as a class

In [6]:
class Player(Base):
    __tablename__ = 'player'
    
    id = Column(Integer, primary_key = True) 
    name = Column(String)
    number = Column(Integer)
    team_id = Column(Integer, ForeignKey("teams.id")) 
    
    def __init__(self, name, number, team=None):
        self.name = name
        self.number = number
        self.team = team

    def __repr__(self):
        return "<Player('%s', '%s')>" % (self.name, self.number)

In [7]:
# Create a table for Players
# with 4 columns: ID, Name, Number, and Team ID
class Player(Base):
    __tablename__ = 'players'
    ## At first, only specify data types for the columns
    ## primary_key is unique, non-nullable identifier for row
    ## Have an ID column because player attributes (name, etc) are not unique
    ## At least 1 primary_key per table
    id = Column(Integer, primary_key = True) 
    name = Column(String)
    number = Column(Integer)
    ## ForeignKey tells us we have a relationship with another table ("teams") by the ("id") variable
    ## This info constrained to only come from that table
    ## What we are referencing is usually the primary key for that table
    team_id = Column(Integer, ForeignKey("teams.id")) 
    ## Populate non-ID fields in the table through the constructor
    def __init__(self, name, number, team=None):
        self.name = name
        self.number = number
        self.team = team
    # Optional to define the representation (what gets returned when you enter this object)
    def __repr__(self):
        return "<Player('%s', '%s')>" % (self.name, self.number)

##### Reviewing the keys on this table

* We defined the column `id` as a primary key for the `players` table . 

- It will uniquely identify a row in the table.
- a foreign key (here `team_id`) will link two tables together by referencing the primary key of the related table. 
- A primary key *cannot* take on NULL values, where foreign keys *can*. 

##### Defining the `Team` table as a class

In [8]:
from sqlalchemy.orm import relationship

In [9]:
class Team(Base):
  __tablename__ = "teams"
  
  id = Column(Integer, primary_key=True)
  name = Column(String)

  players = relationship("Player", backref="team")
  
  def __init__(self, name):
    self.name = name
  
  def __repr__(self):
    return "<team('%s')>" % (self.name)

In [11]:
# Our Team table has 2 columns: ID and Name
class Team(Base):
  __tablename__ = "teams"
  
  id = Column(Integer, primary_key=True)
  name = Column(String)

  ## - relationship() tells us another table wants to reference us
  ## - now notice we use "Player" object syntax and "team" member variable syntax
  ## - Note: this is NOT a column
  ##          but we can call <team obj>.players
  ##          or <player obj>.team
  players = relationship("Player", backref="team")
  
  def __init__(self, name):
    self.name = name
  
  def __repr__(self):
    return "<team('%s')>" % (self.name)

#### 4. Issue commands to database

- Create tables for the first time.
- The `metadata` is a registry which includes the ability to make schema commands to database.
- Our SQLite database does not actually have a `players` table present, so we use `metadata` to issue the SQL "CREATE TABLE" command to the database for all tables that don’t yet exist.

In [12]:
Base.metadata.create_all(engine)

* Now we should see our database "player.db" in our working directory! 

##### Let's check what columns we have in the tables

- SQLAlchemy represents info for specific table with Table attribute

In [13]:
Player.__table__ 

Table('players', MetaData(), Column('id', Integer(), table=<players>, primary_key=True, nullable=False), Column('name', String(), table=<players>), Column('number', Integer(), table=<players>), Column('team_id', Integer(), ForeignKey('teams.id'), table=<players>), schema=None)

In [14]:
Team.__table__

Table('teams', MetaData(), Column('id', Integer(), table=<teams>, primary_key=True, nullable=False), Column('name', String(), table=<teams>), schema=None)

#### 5. Create records for each table

* Very similar syntax to what we've done before with creating class instances
* Let's create one "instance" for each table

In [15]:
p1 = Player(name = "Alma", number = 16)

In [16]:
t1 = Team(name = "WashU")

* **Note**: `p1` and `t1` are not yet connected 

In [17]:
t1.players

[]

* To fix this, add a `team` reference to `p1`  

In [18]:
p1.team = t1

* Now `p1` is part of team object `t1`

In [19]:
t1.players

[<Player('Alma', '16')>]

In [20]:
p1.team

<team('WashU')>

##### Let's create another Player! 

In [21]:
p2 = Player("Masanori", 5)

* Let's print the ID

In [22]:
print(p2.id)

None


##### Nothing...

- Even though we didn’t specify it in the constructor, the `id` attribute still produces a value of `None` when we access it (as opposed to Python’s usual behavior of raising AttributeError for an undefined attribute).
- When we actually write the `p2` object to the db a real ID will be assigned.

#### 6. Create a `Session` to actually store things into the database

* A Session is a "holding zone" for all the objects and associations we have created. 
The Session will hold objects until we commit or close. 

In [23]:
from sqlalchemy.orm import sessionmaker

# sessionmaker object
Session = sessionmaker(engine) 
# Session object
session = Session() 

* Add both players to the session

In [24]:
session.add(p2)
session.add(p1)

##### Let's add multiple players

In [25]:
session.add_all([Player("Abbie", 7),
                 Player("Kuan", 9),
                 Player("Lukas", 21),
                 Player("Federico", 33),
                 Player("Leona", 29),
                 Player("Aaron", 44),
                 Player("Nathan", 17),
                 Player("Frank", 99)])

* See what we've done this session

In [26]:
session.new 

IdentitySet([<Player('Masanori', '5')>, <Player('Alma', '16')>, <team('WashU')>, <Player('Abbie', '7')>, <Player('Kuan', '9')>, <Player('Lukas', '21')>, <Player('Federico', '33')>, <Player('Leona', '29')>, <Player('Aaron', '44')>, <Player('Nathan', '17')>, <Player('Frank', '99')>])

##### Before we commit, if we made a mistake, we can correct it using `session.rollback()` to roll back all changes

* Now, make changes to actual db

In [27]:
session.commit()

##### Now that we created the database, let's make some queries!

* Let's look for ID again
* ID count will preserve the order in which records entered the database

In [28]:
print('ID: ' + str(p2.id))
print('ID: ' + str(p1.id))

ID: 1
ID: 2


* A sample query
    * you can think of the syntax as...`session.query(TABLE).order_by(COLUMN)`

In [29]:
for player in session.query(Player).order_by(Player.number): # .asc() .desc()
  print(player.number, player.name, player.id)

5 Masanori 1
7 Abbie 3
9 Kuan 4
16 Alma 2
17 Nathan 9
21 Lukas 5
29 Leona 7
33 Federico 6
44 Aaron 8
99 Frank 10


##### Using `.filter()`

Source: https://docs.sqlalchemy.org/en/20/tutorial/index.html#unified-tutorial

In [30]:
for player in session.query(Player).filter(Player.name == "Alma").order_by(Player.number):
  print(player.number, player.name)

16 Alma


In [31]:
for player in session.query(Player).filter(Player.name != "Alma").order_by(Player.number):
  print(player.number, player.name)

5 Masanori
7 Abbie
9 Kuan
17 Nathan
21 Lukas
29 Leona
33 Federico
44 Aaron
99 Frank


In [32]:
for player in session.query(Player).filter(Player.number > 10).order_by(Player.number):
  print(player.number, player.name)

16 Alma
17 Nathan
21 Lukas
29 Leona
33 Federico
44 Aaron
99 Frank


##### Using `or_()`

In [33]:
from sqlalchemy import or_

In [34]:
for player in session.query(Player).filter(or_(Player.name == "Alma", 
                                               Player.name == "Leona")).order_by(Player.number):
  print(player.number, player.name)

16 Alma
29 Leona


##### Using `.like()`

* Return all the records where the 'name' column contains a partial string

In [35]:
for player in session.query(Player).filter(Player.name.like("%ri%")).order_by(Player.number):
  print(player.number, player.name)

5 Masanori
33 Federico


##### Using `.and()`

In [36]:
from sqlalchemy import and_
for player in session.query(Player).filter(and_(Player.name.like("%ri%"), Player.number > 21)).order_by(Player.number):
  print(player.number, player.name)

33 Federico


##### Functions

* We can also apply functions using the `func` package

In [37]:
from sqlalchemy import func

* Note that you need to query the columns I want back

In [38]:
for player in session.query(Player.name, Player.number, func.max(Player.number)):
  print(player.name, player.number)

Frank 99


##### Results can be indexed as lists

In [39]:
results = session.query(Player).filter(and_(Player.name.like("%ri%"), 
                                            Player.number > 10)).order_by(Player.number)

In [40]:
results.first()

<Player('Federico', '33')>

* Check the number of elements in results

In [41]:
# len(results)
results.count()

1

In [42]:
results[0]
# results[1]

<Player('Federico', '33')>

Let's create a new team, and assign some players to that team!

In [43]:
duke = Team('Duke')

* Query all players

In [44]:
players = session.query(Player).all()
players

[<Player('Masanori', '5')>,
 <Player('Alma', '16')>,
 <Player('Abbie', '7')>,
 <Player('Kuan', '9')>,
 <Player('Lukas', '21')>,
 <Player('Federico', '33')>,
 <Player('Leona', '29')>,
 <Player('Aaron', '44')>,
 <Player('Nathan', '17')>,
 <Player('Frank', '99')>]

* Assign someone to Duke

In [45]:
p2.team = duke

* Assign someone else

In [46]:
players[2].team = duke
players[2].team

<team('Duke')>

In [47]:
players[3].team = t1
players[4].team = duke
players[5].team = duke
players[6].team = t1

* Check Duke's players

In [48]:
duke.players

[<Player('Masanori', '5')>,
 <Player('Abbie', '7')>,
 <Player('Lukas', '21')>,
 <Player('Federico', '33')>]

In [49]:
t1.players

[<Player('Alma', '16')>, <Player('Kuan', '9')>, <Player('Leona', '29')>]

##### Let's check the players' teams

In [50]:
players[5].team

<team('Duke')>

* We can get the roster from one of the player's team's (other) players

In [51]:
players[5].team.players

[<Player('Masanori', '5')>,
 <Player('Abbie', '7')>,
 <Player('Lukas', '21')>,
 <Player('Federico', '33')>]

* Now note the id

In [52]:
players[5].team_id

2

In [53]:
p1.team_id

1

* Let's commit the changes! 

In [54]:
session.commit()

Load the two tables together using: 

`query(Table1).join(Table2)`

In [55]:
for i in session.query(Player).join(Team).filter(Player.name == "Alma").filter(Team.name == "WashU").order_by(Player.number):
  print(i.number, i.name, i.team.name)

16 Alma WashU


* Equivalently, using `and_()`

In [56]:
for i in session.query(Player).join(Team).filter(and_(Player.name=="Alma", Team.name=="WashU")).order_by(Player.number):
  print(i.number, i.name, i.team.name)

16 Alma WashU


##### Deletion

In [57]:
# list we queried above
players

[<Player('Masanori', '5')>,
 <Player('Alma', '16')>,
 <Player('Abbie', '7')>,
 <Player('Kuan', '9')>,
 <Player('Lukas', '21')>,
 <Player('Federico', '33')>,
 <Player('Leona', '29')>,
 <Player('Aaron', '44')>,
 <Player('Nathan', '17')>,
 <Player('Frank', '99')>]

In [58]:
session.query(Player).filter(Player.number == 16).count()

1

In [59]:
alma = session.query(Player).filter(Player.number == 16).first()

In [60]:
# now we can delete alma
session.delete(alma)

In [61]:
# it's gone!
session.query(Player).filter(Player.number == 16).count()

0

* But it's  still in our object `players`

In [62]:
players

[<Player('Masanori', '5')>,
 <Player('Alma', '16')>,
 <Player('Abbie', '7')>,
 <Player('Kuan', '9')>,
 <Player('Lukas', '21')>,
 <Player('Federico', '33')>,
 <Player('Leona', '29')>,
 <Player('Aaron', '44')>,
 <Player('Nathan', '17')>,
 <Player('Frank', '99')>]

* Let's update our players variable

In [63]:
players = session.query(Player).all()

In [64]:
players

[<Player('Masanori', '5')>,
 <Player('Abbie', '7')>,
 <Player('Kuan', '9')>,
 <Player('Lukas', '21')>,
 <Player('Federico', '33')>,
 <Player('Leona', '29')>,
 <Player('Aaron', '44')>,
 <Player('Nathan', '17')>,
 <Player('Frank', '99')>]

In [65]:
# commit our changes
session.commit()

In [66]:
# print IDs 
[p.id for p in players]

[1, 3, 4, 5, 6, 7, 8, 9, 10]

* Add Alma again

In [67]:
alma = Player(name = "Alma", number = 9)
session.add(alma)

In [68]:
session.commit()

In [69]:
players = session.query(Player).all()

In [70]:
players

[<Player('Masanori', '5')>,
 <Player('Abbie', '7')>,
 <Player('Kuan', '9')>,
 <Player('Lukas', '21')>,
 <Player('Federico', '33')>,
 <Player('Leona', '29')>,
 <Player('Aaron', '44')>,
 <Player('Nathan', '17')>,
 <Player('Frank', '99')>,
 <Player('Alma', '9')>]

* How to convert data to csv

In [78]:
for player in players:
  ## apply skills we've learned already
  print(player.name, player.number, player.team, player.id)

Masanori 5 <team('Duke')> 1
Abbie 7 <team('Duke')> 3
Kuan 9 <team('WashU')> 4
Lukas 21 <team('Duke')> 5
Federico 33 <team('Duke')> 6
Leona 29 <team('WashU')> 7
Aaron 44 None 8
Nathan 17 None 9
Frank 99 None 10
Alma 9 None 11


 * A more complete example

In [72]:
import csv
with open("players.csv", 'w') as f:
    my_writer = csv.DictWriter(f, fieldnames = ("name", "number", "team"))
    my_writer.writeheader()
    for player in players:
        try:
            my_writer.writerow({"name":player.name, "number":player.number, "team":player.team.name})
        except AttributeError:
            my_writer.writerow({"name":player.name, "number":player.number, "team":""})

##### Using pandas

In [74]:
import pandas as pd

In [82]:
player_dict = {'name': [],
              'number': [],
              'team': []}

In [83]:
for player in players:
    player_dict['name'].append(player.name)
    player_dict['number'].append(player.number)
    try:
        player_dict['team'].append(player.team.name)
    except AttributeError:
        player_dict['team'].append("None")

In [84]:
player_df = pd.DataFrame(player_dict)

In [85]:
player_df.head()

,name,number,team
0,Masanori,5,Duke
1,Abbie,7,Duke
2,Kuan,9,WashU
3,Lukas,21,Duke
4,Federico,33,Duke


In [86]:
player_df.to_csv('players2.csv', index=False)

In [73]:
# Copyright of the original version:

# Copyright (c) 2014 Matt Dickenson
# 
# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:
# 
# The above copyright notice and this permission notice shall be included in all
# copies or substantial portions of the Software.
# 
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
# SOFTWARE.